In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import necessary libraries
import os
import sys
import json
import pathlib
import shutil
import random
import tensorflow as tf
import pandas as pd

In [ ]:
sys.path.append(os.path.abspath('/content/drive/MyDrive/code/detection/shared'))

In [ ]:
from xml_to_csv import xml_to_csv

In [ ]:
config = json.load(open('/content/drive/MyDrive/code/detection/training/config.json',))
config

{'anomalous_factor': 20,
 'data_version': 'v1',
 'description': 'The model is trained with data version v1',
 'dispaly_format': 'letter',
 'eval_steps': 500,
 'model_version': 'v1',
 'project_name': 'lentil',
 'remove_anomalies': 'False',
 'test_images': [],
 'test_split': 0.2,
 'threshold': 0.5,
 'train_steps': 10000}

In [ ]:
base_path = '/content/drive/MyDrive/projects'

In [ ]:
base_model_path = os.path.join(base_path, config['project_name'], 'models', config['model_version'])
base_model_path

'/content/drive/MyDrive/projects/lentil/models/v1'

In [ ]:
os.system("rm -rf " + base_model_path)
os.makedirs(base_model_path)

**Setup tensorflow object detection API**

In [ ]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3137, done.
remote: Counting objects: 100% (3137/3137), done.
remote: Compressing objects: 100% (2646/2646), done.
remote: Total 3137 (delta 810), reused 1354 (delta 447), pack-reused 0
Receiving objects: 100% (3137/3137), 33.35 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (810/810), done.


In [ ]:
%%bash
cd /content/models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .

In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 9.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
%%bash
cd /content/models/research
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pyyaml to determine which version is compatible with other r

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


**Split data into train and test sets**

In [ ]:
base_data_path = os.path.join(base_path, config['project_name'], 'data', config['data_version'])
base_data_path

'/content/drive/MyDrive/projects/lentil/data/v1'

In [ ]:
data_config = json.load(open(os.path.join(base_data_path, 'data_config.json'),))
data_config

{'description': "The lentils data created by gomicro using the Coles's specimens",
 'labels': [{'color': 'RoyalBlue', 'id': 1, 'letter': 'S', 'name': 'Stalk'},
  {'color': 'Cyan', 'id': 2, 'letter': 'B', 'name': 'Broken'},
  {'color': 'Magenta', 'id': 3, 'letter': 'P', 'name': 'Pod'},
  {'color': 'OrangeRed', 'id': 4, 'letter': 'S', 'name': 'Seeds'},
  {'color': 'Chartreuse', 'id': 5, 'letter': 'N', 'name': 'Normal'}],
 'patch_overlap': 40,
 'patch_size': 640}

In [ ]:
classes = [None] * len(data_config['labels'])
colors = [None] * len(classes)
letters = [None] * len(classes)

for label in data_config['labels']:
  classes[label['id'] - 1] = label['name']
  colors[label['id'] - 1] = label['color']
  letters[label['id'] - 1] = label['letter']

classes, colors, letters

(['Stalk', 'Broken', 'Pod', 'Seeds', 'Normal'],
 ['RoyalBlue', 'Cyan', 'Magenta', 'OrangeRed', 'Chartreuse'],
 ['S', 'B', 'P', 'S', 'N'])

In [ ]:
!rm -rf data
os.system("unzip -qq " + os.path.join(base_data_path, 'data.zip'))

0

In [ ]:
all_files = os.listdir('data')
xml_files = []
image_files = []

for file_name in all_files:
  if file_name.endswith('.xml'):
    name, extension = os.path.splitext(file_name)
    xml_files.append(name)
  else:
    image_files.append(file_name)

filtered_image_files = []
for file_name in image_files:
  name, extension = os.path.splitext(file_name)
  if name in xml_files:
    filtered_image_files.append(file_name)

len(xml_files), len(filtered_image_files)

(117, 117)

In [ ]:
random.shuffle(filtered_image_files)
train_count = int(len(filtered_image_files) * (1 - config['test_split']))

!rm -rf splits
!mkdir splits
!mkdir splits/train
!mkdir splits/train/images
!mkdir splits/train/labels
!mkdir splits/test
!mkdir splits/test/images
!mkdir splits/test/labels

for idx, file_name in enumerate(filtered_image_files):
  dist = 'train' if idx < train_count else 'test'
  name, extension = os.path.splitext(file_name)
  name, extension = os.path.splitext(file_name)
  shutil.copy(os.path.join('/content/data', file_name), os.path.join('/content/splits', dist, 'images', file_name))    
  shutil.copy(os.path.join('/content/data', name + '.xml'), os.path.join('/content/splits', dist, 'labels', name + '.xml'))

image_path = os.path.join(os.getcwd(), '/content/splits/train/labels')
xml_df = xml_to_csv(image_path, classes)
xml_df.to_csv('/content/splits/train/labels.csv', index=None)

image_path = os.path.join(os.getcwd(), '/content/splits/test/labels')
xml_df = xml_to_csv(image_path, classes)
xml_df.to_csv('/content/splits/test/labels.csv', index=None)

if not pathlib.Path('/content/splits.zip').exists():
  !rm splits.zip
!zip --q -r splits.zip splits

split_path = os.path.join(base_model_path, 'splits.zip')
if pathlib.Path(split_path).exists():
  os.system("rm " + split_path)

shutil.copyfile('/content/splits.zip', split_path)

rm: cannot remove 'splits.zip': No such file or directory


'/content/drive/MyDrive/projects/lentil/models/v1/splits.zip'

In [ ]:
#setup config file and copy to target directory
config['classes'] = classes
config['colors'] = colors
config['letters'] = letters
config['patch_size'] = data_config['patch_size']
config['patch_overlap'] = data_config['patch_overlap']

config_path = os.path.join(base_model_path, 'config.json')
with open(config_path, 'w') as f:
  json.dump(config, f)

**Create tfrecords**

In [ ]:
from generate_tf_record import tf_record_generator

In [ ]:
!rm -rf tf_records
!mkdir tf_records

In [ ]:
generator = tf_record_generator(classes)

def generate_tf_record(phase):
  output_path = os.path.join(os.getcwd(), '/content/tf_records/{0}.record'.format(phase))
  writer = tf.io.TFRecordWriter(output_path)
  path = os.path.join('/content/splits/{0}/images'.format(phase))
  examples = pd.read_csv('/content/splits/{0}/labels.csv'.format(phase))
  grouped = generator.split(examples, 'filename')
  
  for group in grouped:
    tf_example = generator.create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
generate_tf_record('train')
generate_tf_record('test')

Successfully created the TFRecords: /content/tf_records/train.record
Successfully created the TFRecords: /content/tf_records/test.record


In [ ]:
!rm tf_records.zip
!zip -qq -r tf_records.zip tf_records

tf_records_path = os.path.join(base_model_path, 'tf_records.zip')
if pathlib.Path(tf_records_path).exists():
  os.system("rm " + tf_records_path)

shutil.copyfile('/content/tf_records.zip', tf_records_path)

rm: cannot remove 'tf_records.zip': No such file or directory


'/content/drive/MyDrive/projects/lentil/models/v1/tf_records.zip'

**Generate pbtxt file**

In [ ]:
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from google.protobuf import text_format

def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))

    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

In [ ]:
txt = convert_classes(classes)
print(txt)
with open('labels.pbtxt', 'w') as f:
  f.write(txt)

item {
  name: "Stalk"
  id: 1
}
item {
  name: "Broken"
  id: 2
}
item {
  name: "Pod"
  id: 3
}
item {
  name: "Seeds"
  id: 4
}
item {
  name: "Normal"
  id: 5
}



In [ ]:
labels_path = os.path.join(base_model_path, 'labels.pbtxt')
if pathlib.Path(labels_path).exists():
  os.system("rm " + labels_path)

shutil.copyfile('/content/labels.pbtxt', labels_path)

'/content/drive/MyDrive/projects/lentil/models/v1/labels.pbtxt'